In [9]:
from lib.read_data import dataset,Datasets

import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# feature extractors
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

# finetuning
from sklearn.model_selection import GridSearchCV

# validation
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [2]:
def read_data_set(data_table,test_size=0.25,BENCHMARK=False):
    '''
    convert a pandas dataframe data table into Datasets(dataset,dataset)
    '''
    train, test = train_test_split(data_table,test_size=0.25)
    train_x = train[[col for col in train.columns
    if col not in ['INFO','gavin_res']]]
    features = train_x.columns
    train_x = np.array(train_x)
    test_x = np.array(test[[col for col in train.columns
    if col not in ['INFO','gavin_res']]])
    train_y = np.array(train['INFO'],dtype=np.int8)
    test_y = np.array(test['INFO'],dtype=np.int8)

    if BENCHMARK:
        return Datasets(train=dataset(train_x,train_y,features),
                        test=dataset(test_x,test_y,features)),\
                        train['gavin_res'],\
                        test['gavin_res']
    return Datasets(train=dataset(train_x,train_y,features),
                    test=dataset(test_x,test_y,features))

def run_display_output(classifier,test,DRAW=False):
    '''
    get confusion matrix and auc score for test dataset
    (optional) draw roc curve
    '''
    pred = classifier.predict(test.values)
    pred[pred>0.5] = 1
    pred[pred<0.5] = 0
    tn, fp, fn, tp = confusion_matrix(test.labels,pred).ravel()#confusion matrix
    print(tn,fp,fn,tp)
    sensitivity = tp/(fn+tp)
    specificity = tn/(fp+tn)
    prods = classifier.predict(test.values)
    if hasattr(classifier,'predict_proba'):
        prods = classifier.predict_proba(test.values)[:,1]
    else:
        probs = classifier.predict(test.values)
    fpr, tpr, _ = metrics.roc_curve(test.labels, prods)
    score = metrics.auc(fpr,tpr) #auc score
    if DRAW:
        draw_roc_curve(fpr,tpr,score)

    return sensitivity, specificity, score

def display_res_gavin_and_best_model(param_grid,pipeline,mvid,filename=None):
    '''
    use model defined by pipeline to fit mvid Dataset
    gridsearchCV determine the parameters given in param_grid
    (optional) save the model in path given in filename
    '''
    classifier = GridSearchCV(estimator=pipeline,
                              param_grid=param_grid)

    print('Start training...')
    classifier.fit(mvid.train.values,mvid.train.labels)
    print('Model Description:\n',classifier.best_estimator_)
    if filename:
        pickle.dump(classifier,open(filename,'wb'))
        print('Saved model to path:',filename)
    sensitivity,specificity,score = run_display_output(classifier,mvid.test)
    print('>>> best model results: sensitivity: {:.{prec}}\tspecificity: {:.{prec}f}\tauc:{}'.\
    format(sensitivity,specificity,score,prec=3))
    return classifier

def inference(classifier,dataset):
    sensitivity,specificity,score = run_display_output(classifier,dataset.test)
    print('>>> transfer to myo5b gene: sensitivity: {:.{prec}}\tspecificity: {:.{prec}f}\tauc:{}'.\
    format(sensitivity,specificity,score,prec=3))

def read_gavin(gavin_res, labels):
    '''
    compare gavin results with labels for a certain subset of data
    '''
    gavin_res = gavin_res.replace('Pathogenic',1)
    gavin_res = gavin_res.replace('Benign',0)
    tn_g, fp_g, fn_g, tp_g = \
    confusion_matrix(labels, gavin_res.astype(np.int8)).ravel()
    sensitivity_g = tp_g/(fn_g+tp_g)
    specificity_g = tn_g/(fp_g+tn_g)
    return sensitivity_g, specificity_g

In [7]:
if __name__=='__main__':

    # read data
    processed_all = pd.read_csv('data/myh7/myh7_myo5b.csv',sep='\t')
    processed_myh7 = processed_all.loc[processed_all['genename']=='MYH7']
    processed_myo5b = processed_all.loc[processed_all['genename']=='MYO5B']
    processed_myo5b = processed_myo5b.drop('genename',axis=1) # drop pos
    processed_myh7 = processed_myh7.drop('genename',axis=1) # drop pos
    
    myh7_data = read_data_set(processed_myh7,BENCHMARK=False)
    myo5b_data = read_data_set(processed_myo5b,BENCHMARK=False)
    # print(data.head())
    # raise NotImplementedError # check the dataset loaded
    print('Dataset loaded.',myh7_data.train.values.shape)

# ================model selection==========================================
    # # PCA + LogisticRegression
    # # Parameters
    n_components = np.arange(10,50,10)
    class_weight = ['balanced']#,{1:2,0:1}]
    param_grid_logr = [{'pca__n_components':n_components,
                   'logr__penalty':['l1'],#'l2'],
                   'logr__C':[2],#3,4,5],
                   'logr__class_weight':class_weight}]
    # pipeline
    pipeline_logr = Pipeline(steps=[('pca',PCA()),
                               ('logr',LogisticRegression())])
    # save model
    filename = os.path.join('model')#,'pca_logr_new.sav')
    # display results
    classifier_logr = display_res_gavin_and_best_model(param_grid_logr,
                                     pipeline_logr,
                                     myh7_data)#,
                                     #filename)
    
    inference(classifier_logr,myo5b_data)
#     # display gavin results
#     sensitivity_g,specificity_g = read_gavin(test_gavin,myh7_data.test.labels)
#     print('>>> gavin model results: sensitivity: {:.{prec}}\tspecificity: {:.{prec}f}'.\
#     format(sensitivity_g,specificity_g,prec=3))

# ==========================================================
    # Linear model + SGDregressor
    # Parameters
    n_components = np.arange(10,50,10)
    loss = ['squared_loss', 'huber',
            'epsilon_insensitive', 
            'squared_epsilon_insensitive']
    penalty = ['l1','l2','elasticnet']
    l1_ratio = [0.15,0.2,0.5,0.8]
    learning_rate = ['constant','optimal','invscaling']
    param_grid_sgd = [{#'pca__n_components':n_components,
                   'sgd__penalty':penalty,
                   'sgd__loss':loss,
                   'sgd__l1_ratio':l1_ratio,
                   'sgd__learning_rate':learning_rate,
                   'sgd__tol':[1e-6],
                   'sgd__warm_start':[True],
                   'sgd__max_iter':[10000],
                   'sgd__eta0':[0.1,0.01,0.5],
                   'sgd__class_weight':['balanced']}]
    # pipeline
    pipeline_sgd = Pipeline(steps=[#('pca',PCA()),
                               ('sgd',SGDClassifier())])
    # save model
    filename = os.path.join('model','pca_logr_new.sav')
    # display results
    classifier_sgd = display_res_gavin_and_best_model(param_grid_sgd,
                                     pipeline_sgd,
                                     myh7_data)#,
                                     #filename)
    inference(classifier_sgd,myo5b_data)


Dataset loaded. (637, 214)
Start training...
Model Description:
 Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=40, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('logr', LogisticRegression(C=2, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])
102 51 14 46
>>> best model results: sensitivity: 0.767	specificity: 0.667	auc:0.7683006535947713
Start training...
Model Description:
 Pipeline(memory=None,
     steps=[('sgd', SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.01, fit_intercept=True, l1_ratio=0.2,
       learning_rate='constant', loss='squared_epsilon_insensitive',
       max_iter=10000, n_iter=None, n_jobs=1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True,

In [13]:
# Here are some classifiers that support the `partial_fit` method
partial_fit_classifiers = {
    'SGD': SGDClassifier(),
    'Perceptron': Perceptron(),
    'NB Multinomial': MultinomialNB(alpha=0.01),
    'Passive-Aggressive': PassiveAggressiveClassifier(),
}
all_classes = np.array([0, 1])
# Main loop : iterate on mini-batches of examples
for cls_name,classifier in partial_fit_classifiers.items():
    for _iter in range(2000):
        values, labels = myh7_data.train.next_batch(100)
        classifier.partial_fit(values,labels,classes=all_classes)
        print(classifier.score(myh7_data.test.values, myh7_data.test.labels))
    
# for i, (X_train_text, y_train) in enumerate(minibatch_iterators):

#     tick = time.time()
#     X_train = vectorizer.transform(X_train_text)
#     total_vect_time += time.time() - tick

#     for cls_name, cls in partial_fit_classifiers.items():
#         tick = time.time()
#         # update estimator with examples in the current mini-batch
#         cls.partial_fit(X_train, y_train, classes=all_classes)

#         # accumulate test accuracy stats
#         cls_stats[cls_name]['total_fit_time'] += time.time() - tick
#         cls_stats[cls_name]['n_train'] += X_train.shape[0]
#         cls_stats[cls_name]['n_train_pos'] += sum(y_train)
#         tick = time.time()
#         cls_stats[cls_name]['accuracy'] = cls.score(X_test, y_test)
#         cls_stats[cls_name]['prediction_time'] = time.time() - tick
#         acc_history = (cls_stats[cls_name]['accuracy'],
#                        cls_stats[cls_name]['n_train'])
#         cls_stats[cls_name]['accuracy_history'].append(acc_history)
#         run_history = (cls_stats[cls_name]['accuracy'],
#                        total_vect_time + cls_stats[cls_name]['total_fit_time'])
#         cls_stats[cls_name]['runtime_history'].append(run_history)

#         if i % 3 == 0:
#             print(progress(cls_name, cls_stats[cls_name]))
#     if i % 3 == 0:
#         print('\n')

/Users/gcc/.pyenv/versions/py3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.723004694836
0.37558685446
0.708920187793
0.68544600939
0.704225352113
0.713615023474
0.704225352113
0.699530516432
0.680751173709
0.68544600939
0.657276995305
0.723004694836
0.68544600939
0.643192488263
0.413145539906
0.713615023474
0.732394366197
0.713615023474
0.737089201878
0.732394366197
0.727699530516
0.732394366197
0.422535211268
0.713615023474
0.746478873239
0.671361502347
0.732394366197
0.727699530516
0.746478873239
0.737089201878
0.713615023474
0.671361502347
0.746478873239
0.718309859155
0.737089201878
0.56338028169
0.708920187793
0.661971830986
0.718309859155
0.535211267606
0.516431924883
0.713615023474
0.737089201878
0.75117370892
0.727699530516
0.69014084507
0.704225352113
0.661971830986
0.746478873239
0.661971830986
0.741784037559
0.694835680751
0.737089201878
0.699530516432
0.516431924883
0.676056338028
0.727699530516
0.718309859155
0.56338028169
0.694835680751
0.708920187793
0.474178403756
0.741784037559
0.732394366197
0.727699530516
0.713615023474
0.718309859155
0.6

0.652582159624
0.661971830986
0.652582159624
0.741784037559
0.474178403756
0.657276995305
0.713615023474
0.68544600939
0.68544600939
0.676056338028
0.596244131455
0.600938967136
0.723004694836
0.647887323944
0.737089201878
0.68544600939
0.661971830986
0.718309859155
0.741784037559
0.741784037559
0.718309859155
0.708920187793
0.68544600939
0.723004694836
0.741784037559
0.75117370892
0.75117370892
0.723004694836
0.737089201878
0.708920187793
0.75117370892
0.746478873239
0.657276995305
0.765258215962
0.732394366197
0.746478873239
0.586854460094
0.718309859155
0.732394366197
0.713615023474
0.746478873239
0.746478873239
0.652582159624
0.732394366197
0.746478873239
0.75117370892
0.708920187793
0.704225352113
0.741784037559
0.741784037559
0.75117370892
0.737089201878
0.727699530516
0.741784037559
0.746478873239
0.75117370892
0.741784037559
0.737089201878
0.746478873239
0.694835680751
0.62441314554
0.741784037559
0.741784037559
0.699530516432
0.746478873239
0.741784037559
0.746478873239
0.7370

0.746478873239
0.727699530516
0.727699530516
0.732394366197
0.760563380282
0.732394366197
0.732394366197
0.708920187793
0.732394366197
0.723004694836
0.68544600939
0.75117370892
0.619718309859
0.760563380282
0.723004694836
0.732394366197
0.723004694836
0.737089201878
0.732394366197
0.741784037559
0.732394366197
0.732394366197
0.718309859155
0.732394366197
0.755868544601
0.741784037559
0.741784037559
0.741784037559
0.746478873239
0.741784037559
0.718309859155
0.732394366197
0.699530516432
0.718309859155
0.75117370892
0.713615023474
0.732394366197
0.727699530516
0.62441314554
0.737089201878
0.732394366197
0.737089201878
0.699530516432
0.723004694836
0.732394366197
0.708920187793
0.713615023474
0.741784037559
0.723004694836
0.741784037559
0.694835680751
0.741784037559
0.723004694836
0.671361502347
0.704225352113
0.629107981221
0.713615023474
0.713615023474
0.737089201878
0.708920187793
0.643192488263
0.755868544601
0.755868544601
0.708920187793
0.755868544601
0.737089201878
0.75117370892


0.746478873239
0.741784037559
0.741784037559
0.741784037559
0.732394366197
0.760563380282
0.760563380282
0.75117370892
0.718309859155
0.765258215962
0.713615023474
0.694835680751
0.708920187793
0.737089201878
0.737089201878
0.75117370892
0.727699530516
0.737089201878
0.727699530516
0.75117370892
0.737089201878
0.737089201878
0.723004694836
0.68544600939
0.732394366197
0.760563380282
0.741784037559
0.737089201878
0.732394366197
0.704225352113
0.741784037559
0.741784037559
0.572769953052
0.765258215962
0.746478873239
0.732394366197
0.741784037559
0.69014084507
0.737089201878
0.69014084507
0.694835680751
0.755868544601
0.713615023474
0.727699530516
0.610328638498
0.760563380282
0.732394366197
0.75117370892
0.732394366197
0.713615023474
0.741784037559
0.741784037559
0.591549295775
0.713615023474
0.746478873239
0.741784037559
0.746478873239
0.75117370892
0.741784037559
0.694835680751
0.718309859155
0.746478873239
0.746478873239
0.746478873239
0.75117370892
0.643192488263
0.732394366197
0.75

...

0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.74178403

0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.741784037559
0.74178403

0.718309859155
0.69014084507
0.755868544601
0.75117370892
0.727699530516
0.741784037559
0.661971830986
0.723004694836
0.732394366197
0.62441314554
0.723004694836
0.732394366197
0.633802816901
0.666666666667
0.75117370892
0.75117370892
0.75117370892
0.69014084507
0.694835680751
0.755868544601
0.741784037559
0.652582159624
0.558685446009
0.723004694836
0.727699530516
0.718309859155
0.629107981221
0.741784037559
0.629107981221
0.741784037559
0.699530516432
0.723004694836
0.746478873239
0.732394366197
0.718309859155
0.699530516432
0.507042253521
0.737089201878
0.741784037559
0.657276995305
0.755868544601
0.638497652582
0.741784037559
0.727699530516
0.737089201878
0.760563380282
0.737089201878
0.75117370892
0.746478873239
0.765258215962
0.49765258216
0.755868544601
0.741784037559
0.723004694836
0.713615023474
0.718309859155
0.746478873239
0.474178403756
0.652582159624
0.727699530516
0.464788732394
0.732394366197
0.746478873239
0.591549295775
0.760563380282
0.699530516432
0.680751173709
0.69

0.704225352113
0.723004694836
0.727699530516
0.737089201878
0.737089201878
0.62441314554
0.737089201878
0.619718309859
0.694835680751
0.708920187793
0.746478873239
0.713615023474
0.671361502347
0.69014084507
0.723004694836
0.49765258216
0.657276995305
0.746478873239
0.727699530516
0.737089201878
0.713615023474
0.718309859155
0.75117370892
0.671361502347
0.638497652582
0.737089201878
0.755868544601
0.694835680751
0.727699530516
0.699530516432
0.727699530516
0.657276995305
0.746478873239
0.75117370892
0.741784037559
0.737089201878
0.68544600939
0.727699530516
0.535211267606
0.713615023474
0.713615023474
0.727699530516
0.741784037559
0.676056338028
0.727699530516
0.676056338028
0.741784037559
0.699530516432
0.694835680751
0.483568075117
0.741784037559
0.75117370892
0.629107981221
0.718309859155
0.75117370892
0.75117370892
0.723004694836
0.732394366197
0.671361502347
0.746478873239
0.544600938967
0.586854460094
0.737089201878
0.732394366197
0.75117370892
0.652582159624
0.727699530516
0.732

0.741784037559
0.49765258216
0.746478873239
0.741784037559
0.727699530516
0.760563380282
0.741784037559
0.727699530516
0.737089201878
0.723004694836
0.699530516432
0.727699530516
0.699530516432
0.741784037559
0.741784037559
0.638497652582
0.727699530516
0.732394366197
0.68544600939
0.732394366197
0.694835680751
0.75117370892
0.741784037559
0.75117370892
0.680751173709
0.666666666667
0.699530516432
0.647887323944
0.75117370892
0.755868544601
0.732394366197
0.713615023474
0.741784037559
0.746478873239
0.699530516432
0.746478873239
0.737089201878
0.699530516432
0.718309859155
0.741784037559
0.741784037559
0.732394366197
0.610328638498
0.619718309859
0.680751173709
0.765258215962
0.765258215962
0.727699530516
0.69014084507
0.704225352113
0.75117370892
0.699530516432
0.741784037559
0.75117370892
0.727699530516
0.723004694836
0.600938967136
0.62441314554
0.732394366197
0.732394366197
0.732394366197
0.75117370892
0.741784037559
0.75117370892
0.723004694836
0.713615023474
0.732394366197
0.6995